# Proyecto Final Machine Learning

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats
import pickle

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from scipy.optimize import curve_fit

import warnings
warnings.filterwarnings("ignore")

sns.set_style("whitegrid")

## Carga y Limpieza de datos

In [ ]:
''' Se cargan los datos '''
df = pd.read_csv('AirQualityUCI.csv', sep=';')
df.drop(columns=['Unnamed: 15','Unnamed: 16'], inplace=True)
df.dropna(inplace=True)

''' Se cambia al formato deseado '''
cols = [col  for col in df.columns if df[col].dtype == 'O' and col != 'Time']
df['Time'] = df['Time'].str.replace('.',':')
df[cols] = df[cols].replace(',','.',regex = True)
df[df.columns[2:]] = df[df.columns[2:]].astype(float)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Time'] = pd.to_timedelta(df['Time'], errors = 'ignore')

''' Indice como fecha y los valores -200 se pasan a nan'''
df.index = df['Date'] + df['Time']
df.drop(['Date','Time'],axis=1,inplace=True)
df = df.replace(-200,np.nan)
df.index.name = 'Fecha'
df.head()

## Datos Faltantes

In [ ]:
import missingno as msno

In [ ]:
""" Cuantos nan hay por columna """
df.isna().sum()

In [ ]:
''' Se quitan las columnas con demasiados nan '''
df.drop(['NMHC(GT)'],axis=1,inplace=True)

In [ ]:
''' Distribucion de los nan '''
fig =msno.matrix(df[df.isnull().sum().index], labels=True, sparkline=False, figsize=(10,5), fontsize=10)
fig_copy = fig.get_figure()
plt.show()

In [ ]:
def plot_cols(col1,col2):
    '''Plot de columna 2 versus columna 1 '''
    df_nan = df[[col1,col2]].dropna()
    plt.figure(figsize=(15,7))
    plt.plot(df_nan.values.T[0],df_nan.values.T[1],'o')
    plt.xlabel(col1,fontsize=15)
    plt.ylabel(col2,fontsize=15)
    plt.title('Comparacion '+col2+' con '+col1,fontsize=15)

Se usara la relacion entre las columnas GT y PT para rellenar los datos faltantes (siempre que se pueda).

In [ ]:
plot_cols('CO(GT)','PT08.S1(CO)')
plot_cols('NOx(GT)','PT08.S3(NOx)')
plot_cols('NO2(GT)','PT08.S4(NO2)')

Al observar los graficos se ve que para las primeras dos comparaciones es favorable ajustar una curva a los datos.

In [ ]:
def Regresion_Lineal(columnas):
    df_nan = df[columnas].dropna() 
    X = df_nan[[columnas[0]]].values
    y = df_nan[columnas[1]].values
    regr = linear_model.RidgeCV().fit(X,y)
    return regr

def curve(x,a,b,c):
    return a*np.exp(-b*x)+c

def Regresion_Nolineal(columnas):
    df_nan = df[columnas].dropna() 
    X = df_nan[columnas[0]].values
    y = df_nan[columnas[1]].values
    regr,cov = curve_fit(curve,X,y)
    return regr

In [ ]:
modeloCO = Regresion_Lineal(['CO(GT)','PT08.S1(CO)'])
modeloNOx = Regresion_Nolineal(['NOx(GT)','PT08.S3(NOx)'])

In [ ]:
def plot_fit(col1,col2,modelo):
    '''Plot del ajuste del modelo, para columna 2 versus columna 1 '''
    fig = plt.figure(figsize=(16,6))
    for i in range(len(col1)):
        df_nan = df[[col1[i],col2[i]]].dropna()
        X = df_nan.values[:,0].reshape(-1,1)
        try:
            y_pred = modelo[i].predict(X)
            tipo = ''
        except:
            y_pred = curve(X,*modelo[i])
            tipo = '.'
        y = df_nan.values[:,1]
        plt.subplot(1, len(col1), i+1)
        plt.plot(X,y,'o')
        plt.plot(X,y_pred,tipo)
        plt.xlabel(col1[i],fontsize=20)
        plt.ylabel(col2[i],fontsize=20)
        plt.title('Ajuste '+col2[i]+' con respecto a '+col1[i],fontsize=20)
    plt.savefig('imagenes/ajuste.pdf', bbox_inches = 'tight')

In [ ]:
plot_fit(['CO(GT)','NOx(GT)'],['PT08.S1(CO)','PT08.S3(NOx)'],[modeloCO,modeloNOx])

In [ ]:
def rellenar_nan(col1,col2,modelo):
    '''Relleno de datos faltantes dado un modelo ajustado, sino hay modelo se usa fillna de pandas '''
    if col2 == '':
        df[col1] = df[col1].fillna(method = modelo)
    else:
        df_col = df[[col1,col2]]
        indice = df_col[df_col.isnull().any(1)].dropna(axis=0,how='all').index
        hola1 = df_col.loc[indice][[col1]]
        hola2 = df_col.loc[hola1[hola1.isnull().any(1)].index]
        indice_importante = hola2.index
        X = hola2.values[:,1:2]
        try:
            new_df = pd.DataFrame(modelo.predict(X),index = indice_importante,columns = [col1])
        except:
            new_df = pd.DataFrame(curve(X,*modelo),index = indice_importante,columns = [col1])
        df.fillna(new_df[[col1]],inplace=True)

In [ ]:
rellenar_nan('PT08.S1(CO)','CO(GT)',modeloCO)
rellenar_nan('PT08.S3(NOx)','NOx(GT)',modeloNOx)
rellenar_nan(['PT08.S1(CO)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S5(O3)','PT08.S4(NO2)','T','RH','AH'],'','ffill')
df.dropna(axis=1,inplace=True)

In [ ]:
'''Guardar el dataframe en pickle '''
df.to_pickle("el_df.pkl")

In [ ]:
'''Cargar los datos, ya procesados '''
df = pd.read_pickle("el_df.pkl")
df.head()

## Distribución de los datos

In [ ]:
import plotly.graph_objects as go

In [ ]:
def plot_data_distribution(df):
    '''Vista previa de la distribución de los datos, sin considerar la dependencia temporal '''
    
    fig, ax = plt.subplots(nrows=4, ncols=2, figsize=[17, 17])
    fig.tight_layout()

    # list(map(lambda a : a.remove(), ax[-1,1:]))
    fig.suptitle('Distribuciones previa dependencia del tiempo',
                fontsize=20,
                x=0.5,
                y=1.05)

    for axis, col in zip(ax.flatten(), df.columns[:]):
        try :
            sns.distplot(df[col], ax=axis, rug=True)
                
        except RuntimeError:
            sns.distplot(df[col], ax=axis, rug=True, kde=False)
        
        axis.set_xlabel(col, fontsize=15)

    plt.subplots_adjust(wspace=0.4, hspace=0.4)

def plot_tiempo(columna):
    '''Grafico de la columna deseada en funcion del tiempo, usando plotly '''
    fig = go.Figure([go.Scatter(x=df.index, y=df[columna])])
    fig.update_layout(title={'text': f"{columna} en función del tiempo", 'y':0.9, 'x':0.5, 
                             'xanchor': 'center', 'yanchor': 'top'},
                      xaxis_title="Fecha",
                      yaxis_title=f"{columna}",
                      font=dict(size=15),
                      width=900, height=500)
    fig.show()

In [ ]:
'''Distribucion de los datos, sin considerar la dependencia temporal '''
plot_data_distribution(df)

In [ ]:
''' Datos en funcion del tiempo '''
k=0
for col in df.columns:
    if k==5:
        break
    plot_tiempo(col)
    k+=1

## Distribución outliers

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=[17, 17])
fig.tight_layout()
fig.suptitle('Distribucion boxplot de los datos', fontsize=20, x=0.5, y=1.05)
for axis, col in zip(ax.flatten(), df.columns[:]):
    df[col].plot.box(ax = axis)
plt.subplots_adjust(wspace=0.4, hspace=0.4)

In [ ]:
# que hacer con los outliers???

## Análisis de correlación

In [ ]:
df_corr = df.corr()
df_corr
# print(df_corr.iloc[:4,:4].to_latex())

Se puede apreciar que las columnas de interes (concentraciones) tienen una alta correlacion entre ellas, pero no asi con la temperatura y la humedad (NO2 igual un poco)

In [ ]:
'''Diccionario con posibles variables para predecir variables de interes '''
dic_pred = dict()
for x in df_corr.columns:
    dic_pred[x]=[]
    for y in df_corr.columns:
        if abs(df_corr.loc[x,y]) > 0.5:
            dic_pred[x].append(y)
del dic_pred['T'], dic_pred['RH'], dic_pred['AH']
for key,value in dic_pred.items():
    print(f"{key}: {value}")

## Matraca prediccion

Como primer acercamiento, se estudiará el desempeño del modelo ARIMA para la variable CO, que se muestra a continuación.

In [ ]:
plot_tiempo('PT08.S1(CO)')

In [ ]:
df[['PT08.S1(CO)']].describe()

In [ ]:
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
def train_test_split(df,tamaño = 0.8):
    X_train = df[:int(df.shape[0]*(tamaño))]
    X_test = df[int(df.shape[0]*tamaño):]
    return X_train,X_test

X_train, X_test = train_test_split(df)
print(f'X_train: {len(X_train)}')
print(f'X_test: {len(X_test)}')

In [ ]:
X_test.head()

In [ ]:
def Modelo_auto_arima(columna):
    df_train = X_train[columna]
    print('Encontrar Parametros Optimos')
    model = auto_arima(df_train, start_p=24, start_q=6, max_p=24, max_q=6, d=0, max_d=0,
                       seasonal = True,trace=True,error_action='ignore',suppress_warnings=True, stepwise=True)
    print('Fiteo Terminado')
    return model.get_params()

In [ ]:
'''Cargar modelo '''
try:
    model_CO = pickle.load(open('modelos/modelo_CO.pkl','rb'))
except:
    try:
        params_model = pickle.load(open('params_modelo.pkl','rb'))
    except:
        params_model = Modelo_auto_arima('PT08.S1(CO)')
    model_CO = SARIMAX(X_train['PT08.S1(CO)'], **params_model)
    model_CO = model_CO.fit()
    print(model_CO.summary())
    pickle.dump(model_CO, open('modelos/modelo_CO.pkl','wb'))

In [ ]:
def error_pred(test, prediccion):
    '''Retorna el Mean Absolute Error y el Mean Squared Error de la prediccion versus el testeo'''
    mae = np.mean(abs(test-prediccion))
    mse = np.mean((test-prediccion)**2)
    return mae, mse

def plot_ajuste(modelo, datos, tipo = 'entrenamiento', largo=300):
    '''Plotea la prediccion para el conjunto de entrenamiento o testeo, para {largo} primeros datos '''
    name = datos.name
    plt.figure(figsize=(15,7))
    plt.xlabel('Fecha', fontsize=20)
    plt.ylabel(name, fontsize=20)
    plt.title(f'Predicción de {name} en conjunto de {tipo}', fontsize=20)

    if tipo == 'entrenamiento':
        pred = modelo.predict()
        plt.plot(datos[1:largo], label = 'Valor Real')
        plt.plot(pred[1:largo],'o-', label = 'Prediccion')

    elif tipo == 'test':
        model_v2 = modelo.extend(datos)
        pred = model_v2.predict()
        plt.plot(datos[:largo], label = 'Valor Real')
        plt.plot(pred[:largo], 'o-', label = 'Prediccion')
        
    plt.legend(loc='best', fontsize=17)
    plt.savefig(f'imagenes/prediccion_{name}_{tipo}_{largo}.pdf', bbox_inches = 'tight')
    plt.show()
    
    '''Tabla errores '''
    mae, mse = error_pred(datos, pred)
    error = pd.DataFrame([mae, mse], index= ['MAE','MSE'], columns=[f'Error {tipo}']).T
    display(error)

def prediccion(modelo, testeo, pasos=24):
    '''Predice la variable una cantidad de pasos al futuro, separada en bloques '''
    y_pred = pd.Series()
    for k in range(int(testeo.shape[0]/pasos)):
        y_test = testeo[pasos*k : pasos*(k+1)]
        pred = modelo.forecast(steps=pasos)
        y_pred = y_pred.append(pred)
        modelo = modelo.extend(y_test)
    return y_pred

def ploteo(testeo, prediccion, pasos = 24, bloques = 4):
    '''Plotea la prediccion y el valor real para pasos en el futuro y una cierta cantidad de bloques de datos '''
    name = testeo.name
    plt.figure(figsize=(15,7))
    plt.xlabel('Fecha', fontsize=20)
    plt.ylabel(name, fontsize=20)
    plt.title(f'Prediccion de {name} para {bloques} bloques de datos', fontsize=20)
    colores = ['r','b','g','y']
    j = 0
    for k in range(bloques):
        y_test = testeo[pasos*k : pasos*(k+1)]
        y_pred = prediccion[pasos*k : pasos*(k+1)]
        if k>=4:
            k = k%4
        if j==0:
            j=1
            plt.plot(y_test,color=colores[k], label = 'Valor Real')
            plt.plot(y_pred,'o-',color=colores[k], label = 'Predicion')
        else: 
            plt.plot(y_test,color=colores[k])
            plt.plot(y_pred, 'o-',color=colores[k])
    plt.legend(loc='best', fontsize=17)
    plt.savefig(f'imagenes/prediccion_{name}_pasos_{pasos}_bloques_{bloques}.pdf', bbox_inches = 'tight')
    plt.show()
    mae, mse = error_pred(testeo, prediccion)
    error = pd.DataFrame([mae, mse], index= ['MAE','MSE'], columns=['Error test']).T
    display(error)

In [ ]:
plot_ajuste(model_CO, X_train['PT08.S1(CO)'], 'entrenamiento', 300)

In [ ]:
plot_ajuste(model_CO, X_test['PT08.S1(CO)'], 'test', 100)

In [ ]:
pasos = 24 # pasos al futuro
bloques = 8
y_pred = prediccion(model_CO, X_test['PT08.S1(CO)'], pasos=pasos)
ploteo(X_test['PT08.S1(CO)'], y_pred, pasos = pasos, bloques = bloques)

## Modelos otras concentraciones

In [ ]:
# variables de interes
# PT08.S1(CO)	PT08.S2(NMHC)	PT08.S3(NOx)	PT08.S4(NO2)	PT08.S5(O3)

### NO2

In [ ]:
'''Cargar modelo '''
try:
    model_NO2 = pickle.load(open('modelos/modelo_NO2.pkl','rb'))
except:
    params_model = pickle.load(open('params_modelo.pkl','rb'))
    model_NO2 = SARIMAX(X_train['PT08.S4(NO2)'], **params_model)
    model_NO2 = model_NO2.fit()
    print(model_NO2.summary())
    pickle.dump(model_NO2, open('modelos/modelo_NO2.pkl','wb'))

In [ ]:
plot_ajuste(model_NO2, X_train['PT08.S4(NO2)'], 'entrenamiento', 300)
plot_ajuste(model_NO2, X_test['PT08.S4(NO2)'], 'test', 100)

In [ ]:
pasos = 12 # 12 pasos al futuro
bloques = 4
y_pred_NO2 = prediccion(model_NO2, X_test['PT08.S4(NO2)'], pasos=pasos)
ploteo(X_test['PT08.S4(NO2)'], y_pred_NO2, pasos = pasos, bloques = bloques)

### NOx

In [ ]:
'''Cargar modelo '''
try:
    model_NOx = pickle.load(open('modelos/modelo_NOx.pkl','rb'))
except:
    params_model = pickle.load(open('params_modelo.pkl','rb'))
    model_NOx = SARIMAX(X_train['PT08.S3(NOx)'], **params_model)
    model_NOx = model_NOx.fit()
    print(model_NOx.summary())
    pickle.dump(model_NOx, open('modelos/modelo_NOx.pkl','wb'))

In [ ]:
plot_ajuste(model_NOx, X_train['PT08.S3(NOx)'], 'entrenamiento', 300)
plot_ajuste(model_NOx, X_test['PT08.S3(NOx)'], 'test', 100)

In [ ]:
pasos = 12 # 12 pasos al futuro
bloques = 4
y_pred_NOx = prediccion(model_NOx, X_test['PT08.S3(NOx)'], pasos=pasos)
ploteo(X_test['PT08.S3(NOx)'], y_pred_NOx, pasos = pasos, bloques = bloques)

### NMHC

In [ ]:
'''Cargar modelo '''
try:
    model_NMHC = pickle.load(open('modelos/modelo_NMHC.pkl','rb'))
except:
    params_model = pickle.load(open('params_modelo.pkl','rb'))
    model_NMHC = SARIMAX(X_train['PT08.S2(NMHC)'], **params_model)
    model_NMHC = model_NMHC.fit()
    print(model_NMHC.summary())
    pickle.dump(model_NMHC, open('modelos/modelo_NMHC.pkl','wb'))

In [ ]:
plot_ajuste(model_NMHC, X_train['PT08.S2(NMHC)'], 'entrenamiento', 300)
plot_ajuste(model_NMHC, X_test['PT08.S2(NMHC)'], 'test', 100)

In [ ]:
pasos = 12 # 12 pasos al futuro
bloques = 4
y_pred_NMHC = prediccion(model_NMHC, X_test['PT08.S2(NMHC)'], pasos=pasos)
ploteo(X_test['PT08.S2(NMHC)'], y_pred_NMHC, pasos = pasos, bloques = bloques)